In [3]:
import dotenv
from chat_services.llms.chaopenai import build_llm
from chat_services import ChatArgs

dotenv.load_dotenv()

# Define test chat arguments
chat_args = ChatArgs(
    conversation_id="560dde47-8079-4dc2-b902-70ec11a74b44",
    file_path="./tours/Boston Common - Wikipedia.pdf",
    tourID="24352fcc-cecd-45e0-821d-105437274172", #this tour is for the boston common wiki 24352fcc-cecd-45e0-821d-105437274172
    metadata={
        "conversation_id": "560dde47-8079-4dc2-b902-70ec11a74b44",
        "user_id": "123",
        "tourID": "24352fcc-cecd-45e0-821d-105437274172"
    },
    streaming=False,
    k=10, # returns number of docs from vstore
    score_threshold=0.8
)

dotenv.load_dotenv()
llm = build_llm(chat_args)

# llm.invoke("who are you")

AIMessage(content='I am a language model AI created by OpenAI. I am here to assist you with any questions or tasks you may have. How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 10, 'total_tokens': 43}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8e68d142-4245-4509-a3c1-387256e71938-0', usage_metadata={'input_tokens': 10, 'output_tokens': 33, 'total_tokens': 43})

In [8]:

from chat_services.vector_stores.pinecone import vector_store

retriever = vector_store.as_retriever()
results = vector_store.as_retriever().invoke("Boston Common")

print(results)

[Document(page_content='11/3/23, 8:57 AM Boston Common - Wikipedia\nhttps://en.wikipedia.org/wiki/Boston_Common 6/9\nThe Massachusetts State House\noverlooks part of the Common\nThe Frog Pond \nParkman Bandstand \nMassachusetts State\nHouse/Massachusetts\nStatehouse ("New" State\nHouse)\nBoylston station \nBoston Common \nAerial view of Parkman\nBandstand\nThe Massachusetts State House  stands across Beacon\nStreet from the northern edge of the Common. The Boston Public Garden , a more formal landscaped park,\nlies to the west of the Common across Charles Street (and\nwas originally considered an extension of the Common).', metadata={'filename': './tours/Boston Common - Wikipedia.pdf', 'label': '24352fcc-cecd-45e0-821d-105437274172', 'page': 5.0}), Document(page_content='11/3/23, 8:57 AM Boston Common - Wikipedia\nhttps://en.wikipedia.org/wiki/Boston_Common 4/9\nBoston Street Scene\n(Boston Common),\nEdward Mitchell\nBannister, a depiction of\nthe street and Boston\nCommon area in 1898

In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [9]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [14]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This approach allows for easier understanding and handling of the overall task by dividing it into smaller steps. Task decomposition is often used in project management and problem-solving to improve efficiency and facilitate better organization of work.'

In [18]:
print(store)

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This approach allows for easier understanding and handling of the overall task by dividing it into smaller steps. Task decomposition is often used in project management and problem-solving to improve efficiency and facilitate better organization of work.'), HumanMessage(content='What are common ways of doing it?'), AIMessage(content='Common ways of task decomposition include creating a hierarchical structure of tasks, breaking down the task into sequential steps, identifying dependencies between sub-tasks, and assigning responsibilities for each sub-task to different team members. Additionally, using tools such as task lists, Gantt charts, or Kanban boards can help visually represent the decomposed tasks and track progress effectively.')])}


In [17]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Common ways of task decomposition include creating a hierarchical structure of tasks, breaking down the task into sequential steps, identifying dependencies between sub-tasks, and assigning responsibilities for each sub-task to different team members. Additionally, using tools such as task lists, Gantt charts, or Kanban boards can help visually represent the decomposed tasks and track progress effectively.'

In [19]:
from langchain_core.messages import AIMessage

for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks or components. This approach allows for easier understanding and handling of the overall task by dividing it into smaller steps. Task decomposition is often used in project management and problem-solving to improve efficiency and facilitate better organization of work.

User: What are common ways of doing it?

AI: Common ways of task decomposition include creating a hierarchical structure of tasks, breaking down the task into sequential steps, identifying dependencies between sub-tasks, and assigning responsibilities for each sub-task to different team members. Additionally, using tools such as task lists, Gantt charts, or Kanban boards can help visually represent the decomposed tasks and track progress effectively.

